# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [14]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

In [15]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import SnowballStemmer

from nltk import pos_tag
from collections import defaultdict

In [62]:
#somme global variables
prefixes = ['ante','anti','auto','co','com', 'con','contra', 'contro','de','dis','en','ex','extra','hetero','homo','hyper',
'il', 'im', 'in', 'ir','inter','intra','macro','non','omni','post','pre','pro','pseudo','sub','sym', 'syn','tele','tetra',
'trans','tri','un','uni','up']

prefix_l2 = ['contra', 'contro','extra','hetero','inter','hyper','intra','macro','trans']

Noun_suffixes = ['acy','al','ance','ence','dom','er','ee','or','ism','ist','ity', 'ty','ment','ness','ship','sion','tion']
Verb_Suffixes  = ['ate','en','ify', 'fy','ize', 'ise']
Adjective_Suffixes = ['able', 'ible','al','ary','ed','ing', 'esque','ful','ic', 'ical','ish','ious', 'ous','ive','less','ly']
Adverb_Suffixes = ['ly', 'ward', 'wise']
plural_suffixes = ['es', 's']
plural_sf = {'ies':'y' , 'ves':'f'}

suffixes = Noun_suffixes+Verb_Suffixes+Adjective_Suffixes+Adverb_Suffixes+plural_suffixes
Notusing_suffixes = []

In [70]:
HANGMAN_URL = "https://www.trexsim.com/trexsim/hangman"

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.master_flag = 1
        self.current_dictionary = []
        self.wrong_guesses = []
        #self.strat3_flag = 1
        
    def guess(self, word): # word input example: "_ p p _ e "
        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word[::2].replace("_",".") 
        # find length of passed word
        len_word = len(clean_word)
        
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        nodup_dictionary = []
        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue
                
            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)
                nodup_dictionary.append("".join(set(dict_word)))
                
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        # count occurrence of all characters in possible word matches
        full_dict_string = "".join(nodup_dictionary)    
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common()                   
        
        guess_letter = '!'
        
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break

        return guess_letter
    
    def guess_strat2(self, word, isstat2=True, m=0):
        clean_word = word[::2].replace("_",".")
        if isstat2:
            clean_word_temp = clean_word
            clean_word = clean_word_temp.strip('.')
            m = len(clean_word_temp) - len(clean_word)
            clean_word = '(.*)'+clean_word+'(.*)'
        # find length of passed word
        len_word = len(clean_word)
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        nodup_dictionary = []
        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) > m+1+ len_word or len(dict_word) < len_word:
                continue
            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)
                nodup_dictionary.append("".join(set(dict_word)))   
                
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary        
        # count occurrence of all characters in possible word matches        
        full_dict_string = "".join(nodup_dictionary)
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common() 
        
        guess_letter = '!'
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break  
        
        return guess_letter
        
    def word_clean_st3(self, word):
        clean_word = word[::2].replace("_",".")
        history_len = len(clean_word)
        # prefix cleaning code   
        pref = ""
        for pf in prefixes:
            if clean_word[:len(pf)] == pf and len(pref) < len(pf):  
                pref = pf
        '''for pf in prefix_l2:
            if re.match(clean_word[:len(pf)], pf) and len(clean_word[:len(pf)].replace(".", "")) => 3:
                pref = pf'''
        if pref != "":
            clean_word = clean_word[len(pref):]

        #final depluralization, as it can only occur in the last place
        suff = ""
        for sf in plural_sf:
            if clean_word[-len(sf):] == sf and len(suff) < len(sf):
                suff = sf
        if suff != "":
            clean_word = clean_word[:-len(suff)] + plural_sf[suff]
        #suffix cleaning code 
        suff = "./"
        while len(suff)>0:
            suff = ""
            for sf in suffixes:
                if clean_word[-len(sf):] == sf and len(suff) < len(sf):
                    suff = sf
            if suff != "":
                clean_word = clean_word[:-len(suff)]
        #final word
        word = " ".join(clean_word).replace(".","_") + ' '
        return word, history_len- len(clean_word)
    
    def guess_strat4(self, word):
        clean_word = word[::2].replace("_",".")
        guess_letter = '!'
        if clean_word.count('.') <= 2 and clean_word[-1] == '.':
            if clean_word[-2] in ('l', 'c', 'h') and 'y' not in self.guessed_letters:
                guess_letter = 'y'
            elif clean_word[-2] == 'e' and 'd' not in self.guessed_letters:
                guess_letter = 'd'
            elif 's' not in self.guessed_letters:
                guess_letter = 's'
        if guess_letter =='!':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break      
        else:
            print("Strat4 magic happened")
        return guess_letter
    
    def delete_irrelevent_words(self, letter):
        current_dictionary = self.current_dictionary
        #del_list = [word for word in current_dictionary if letter in word]
                
        final_dictionary = [wd for wd in current_dictionary if letter not in wd]
        return final_dictionary
    
    def build_dictionary(self,dictionary_file_location):
        print(" Building dictionary, will take 0.41 sec/1000 words")
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()

        lemmatizer = WordNetLemmatizer()
        snowball = SnowballStemmer(language='english')
        tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

        fuller_dictionary = []
        for word in full_dictionary:
            tk, tg = pos_tag([word])[0]
            lem =lemmatizer.lemmatize(tk, tag_dict.get(tg[0], wordnet.NOUN))
            stm = snowball.stem(word)

            if stm != word and word != lem:
                lem = stm
            sing = word
            for sf in plural_sf:
                if word[-len(sf):] == sf:
                    sing = word[:-len(sf)] + plural_sf[sf]
    
            fuller_dictionary += list(set([word,stm,lem, sing]))
        print(" Removing Duplicates")
        fuller_dictionary = list(dict.fromkeys(fuller_dictionary))
        return fuller_dictionary
    
    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
        self.master_flag = 1
        self.wrong_guesses = []   
        #self.strat3_flag = 1
        
        response = self.request("/new_game", {"practice":practice})

        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                if self.master_flag ==1:
                    guess_letter = self.guess(word)
                    if guess_letter == '!':
                        print("Strat 1 returned no solution, changing to strat 2.")
                        self.master_flag = 2
                        self.current_dictionary = self.full_dictionary

                if self.master_flag ==2:
                    guess_letter = self.guess_strat2(word,True)
                    if guess_letter == '!':
                        print("Strat 2 returned no solution, changing to strat 3.")
                        self.master_flag = 3
                        self.current_dictionary = self.full_dictionary

                if self.master_flag ==3:
                    new_word, m = self.word_clean_st3(word)
                    if new_word != word:
                        print("using new_word from strat3 reduction:", new_word)
                        guess_letter = self.guess_strat2(new_word, False,m) #strat3
                        if  guess_letter == '!':
                            print("Strat 3 returned no solution, changing to strat 4.")
                            self.master_flag = 4
                            self.current_dictionary = self.full_dictionary
                    else:
                        print("Strat 3 not useful now, changing to strat 4.")
                        self.master_flag = 4
                        self.current_dictionary = self.full_dictionary

                if self.master_flag ==4:
                    new_word, m = word, 0
                    if word.count('_')/len(word) < 0.1:
                        new_word, m = self.word_clean_st3(word)
                    guess_letter = '!'
                    if new_word != word:
                        print("using new_word in clean3 reduction:", new_word)
                        guess_letter = self.guess_strat2(new_word, False,m) #strat3
                    if guess_letter == '!':
                        print("using full_word in strat2:", word)
                        self.current_dictionary = self.full_dictionary
                        guess_letter = self.guess_strat2(word,True) #strat2
                    if guess_letter == '!':
                        print("Nothing worked!!")
                        guess_letter = self.guess_strat4(word)

                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Server response: {0}".format(res))
                
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return (True, self.master_flag)
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return (False, self.master_flag)
                elif status=="ongoing":
                    if res.get('word') == word and self.master_flag == 1: # that is our guess is wrong
                        self.wrong_guesses.append(guess_letter)
                        self.current_dictionary = self.delete_irrelevent_words(guess_letter)
                    if len(self.current_dictionary) == 0 and self.master_flag < 4:
                        self.master_flag += 1
                        print("Current Scope is empty, changing to strat{0}".format(self.master_flag))
                        self.current_dictionary = self.full_dictionary
                    # uppdating the word
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return (status=="success", 0)
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        num_retry, time_sleep = 5, 2                                                                                        
        for it in range(num_retry):                                                                                         
            try:                                                                                                            
                response = self.session.request(                                                                            
                    method or "GET",                                                                                        
                    HANGMAN_URL + path,                                                                                     
                    timeout=self.timeout,                                                                                   
                    params=args,                                                                                            
                    data=post_args                                                                                          
                )                                                                                                           
                break                                                                                                       
            except requests.HTTPError as e:                                                                                 
                response = json.loads(e.read())                                                                             
                raise HangmanAPIError(response)                                                                             
            except requests.exceptions.SSLError as e:                                                                       
                if it + 1 == num_retry:                                                                                     
                    raise                                                                                                   
                time.sleep(time_sleep)  

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [71]:
api = HangmanAPI(access_token="16d04ed2733fd37b75934590e1186f", timeout=2000)


 Building dictionary, will take 0.41 sec/1000 words
 Removing Duplicates


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [76]:
practice_counter= 0
practice_success = [[0,0,0,0,0], [0,0,0,0,0]] #fisrt is failure array and second is sucess array
for i in range(50):
    print("Playing game no: ", i)
    (res, desc) = api.start_game(practice=1,verbose=True)
    practice_counter += res
    practice_success[res][desc] += 1
#time.sleep(0.2)
'''[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))
'''
print('counter ', counter, '\n')

#print("Total wins:" , counter)

Successfully start a new game! Game ID: c104d8c16933. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': 'c104d8c16933', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ e _ '}
Guessing letter: r
Server response: {'game_id': 'c104d8c16933', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ e _ '}
Guessing letter: s
Server response: {'game_id': 'c104d8c16933', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ e s '}
Guessing letter: i
Server response: {'game_id': 'c104d8c16933', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ i _ _ e s '}
Guessing letter: n
Server response: {'game_id': 'c104d8c16933', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ i _ _ e s '}
Guessing letter: l
Server response: {'game_id': 'c104d8c16933', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ _ i _ _ e s '}
Guessing letter: a
Server response: {'game_id': 'c104d8c16933', 'status': 'ongoing', 'tries_remains

Server response: {'game_id': '014c45a50eea', 'status': 'ongoing', 'tries_remains': 3, 'word': 'g r a _ _ m e n '}
Current Scope is empty, changing to strat2
Strat 2 returned no solution, changing to strat 3.
using new_word from strat3 reduction: g r a _ _ m 
Guessing letter: t
Server response: {'game_id': '014c45a50eea', 'status': 'ongoing', 'tries_remains': 2, 'word': 'g r a _ _ m e n '}
using new_word from strat3 reduction: g r a _ _ m 
Guessing letter: h
Server response: {'game_id': '014c45a50eea', 'status': 'ongoing', 'tries_remains': 1, 'word': 'g r a _ _ m e n '}
using new_word from strat3 reduction: g r a _ _ m 
Guessing letter: d
Server response: {'game_id': '014c45a50eea', 'status': 'failed', 'tries_remains': 0, 'word': 'g r a _ _ m e n '}
Failed game: 014c45a50eea. Because of: # of tries exceeded!
Successfully start a new game! Game ID: 566fecb02236. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': '566fecb02236', 'status': 

Server response: {'game_id': '175609fdabb9', 'status': 'ongoing', 'tries_remains': 3, 'word': 'a _ _ _ i _ _ i _ _ a l '}
using new_word from strat3 reduction: a _ _ _ i _ _ i _ _ 
Guessing letter: o
Server response: {'game_id': '175609fdabb9', 'status': 'ongoing', 'tries_remains': 2, 'word': 'a _ _ _ i _ _ i _ _ a l '}
using new_word from strat3 reduction: a _ _ _ i _ _ i _ _ 
Guessing letter: c
Server response: {'game_id': '175609fdabb9', 'status': 'ongoing', 'tries_remains': 2, 'word': 'a _ _ _ i c _ i _ _ a l '}
using new_word from strat3 reduction: a _ _ _ i c _ i _ _ 
Guessing letter: f
Server response: {'game_id': '175609fdabb9', 'status': 'ongoing', 'tries_remains': 1, 'word': 'a _ _ _ i c _ i _ _ a l '}
using new_word from strat3 reduction: a _ _ _ i c _ i _ _ 
Guessing letter: s
Server response: {'game_id': '175609fdabb9', 'status': 'failed', 'tries_remains': 0, 'word': 'a _ _ _ i c _ i _ _ a l '}
Failed game: 175609fdabb9. Because of: # of tries exceeded!
Successfully start 

Server response: {'game_id': '1806632fdc27', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ _ _ _ _ i _ e '}
Guessing letter: n
Server response: {'game_id': '1806632fdc27', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ _ _ n _ i _ e '}
Guessing letter: c
Server response: {'game_id': '1806632fdc27', 'status': 'ongoing', 'tries_remains': 1, 'word': '_ _ _ n _ i _ e '}
Guessing letter: s
Server response: {'game_id': '1806632fdc27', 'status': 'ongoing', 'tries_remains': 1, 'word': 's _ _ n _ i _ e '}
Guessing letter: v
Server response: {'game_id': '1806632fdc27', 'status': 'failed', 'tries_remains': 0, 'word': 's _ _ n _ i _ e '}
Failed game: 1806632fdc27. Because of: # of tries exceeded!
Successfully start a new game! Game ID: cd8973daa147. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': 'cd8973daa147', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ _ _ _ e _ '}
Guessing letter: r
Server response: {'game_

Server response: {'game_id': 'd5ad9345eb9a', 'status': 'ongoing', 'tries_remains': 1, 'word': 'r _ n _ _ _ a _ e r '}
using new_word from strat3 reduction: r _ n _ _ _ a _ 
Guessing letter: h
Server response: {'game_id': 'd5ad9345eb9a', 'status': 'failed', 'tries_remains': 0, 'word': 'r _ n _ _ _ a _ e r '}
Failed game: d5ad9345eb9a. Because of: # of tries exceeded!
Successfully start a new game! Game ID: 0610be748462. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': '0610be748462', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ e _ '}
Guessing letter: r
Server response: {'game_id': '0610be748462', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ e _ '}
Guessing letter: s
Server response: {'game_id': '0610be748462', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ _ _ _ e _ '}
Guessing letter: d
Server response: {'game_id': '0610be748462', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ d

...

Server response: {'game_id': 'dcdd5114b369', 'status': 'ongoing', 'tries_remains': 6, 'word': 'p r _ t e _ _ e '}
Guessing letter: o
Server response: {'game_id': 'dcdd5114b369', 'status': 'ongoing', 'tries_remains': 6, 'word': 'p r o t e _ _ e '}
Guessing letter: s
Server response: {'game_id': 'dcdd5114b369', 'status': 'ongoing', 'tries_remains': 5, 'word': 'p r o t e _ _ e '}
Guessing letter: g
Server response: {'game_id': 'dcdd5114b369', 'status': 'ongoing', 'tries_remains': 4, 'word': 'p r o t e _ _ e '}
Current Scope is empty, changing to strat2
Guessing letter: a
Server response: {'game_id': 'dcdd5114b369', 'status': 'ongoing', 'tries_remains': 3, 'word': 'p r o t e _ _ e '}
Guessing letter: d
Server response: {'game_id': 'dcdd5114b369', 'status': 'ongoing', 'tries_remains': 3, 'word': 'p r o t e _ d e '}
Strat 2 returned no solution, changing to strat 3.
using new_word from strat3 reduction: t e _ d e 
Guessing letter: n
Server response: {'game_id': 'dcdd5114b369', 'status': 'ong

Successfully start a new game! Game ID: e06857bea414. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': 'e06857bea414', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ e _ _ _ _ _ e '}
Guessing letter: i
Server response: {'game_id': 'e06857bea414', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ e _ _ _ _ _ e '}
Guessing letter: a
Server response: {'game_id': 'e06857bea414', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ e _ _ a _ _ e '}
Guessing letter: l
Server response: {'game_id': 'e06857bea414', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ e _ _ a _ l e '}
Guessing letter: b
Server response: {'game_id': 'e06857bea414', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ e _ _ a b l e '}
Guessing letter: s
Server response: {'game_id': 'e06857bea414', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ e _ _ a b l e '}
Guessing letter: d
Server response: {'game_id': 'e06857bea414', 'status': 'ongoing', 'tries_remains

Server response: {'game_id': 'a91ba98889ea', 'status': 'ongoing', 'tries_remains': 4, 'word': 'p r o c o n _ c r i p t i o n '}
using new_word in clean3 reduction: c o n _ c r i p 
using full_word in strat2: p r o c o n _ c r i p t i o n 
Nothing worked!!
Guessing letter: s
Server response: {'game_id': 'a91ba98889ea', 'status': 'success', 'tries_remains': 4, 'word': 'p r o c o n s c r i p t i o n '}
Successfully finished game: a91ba98889ea
Successfully start a new game! Game ID: 094933617c8f. # of tries remaining: 6. Word: _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': '094933617c8f', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ e _ _ _ '}
Guessing letter: r
Server response: {'game_id': '094933617c8f', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ e _ _ _ '}
Guessing letter: n
Server response: {'game_id': '094933617c8f', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ e _ _ _ '}
Guessing letter: l
Server response: {'game_id': '094933617c8f'

Server response: {'game_id': '6e737ad1119c', 'status': 'ongoing', 'tries_remains': 4, 'word': 'w _ l l s e n _ '}
Strat 1 returned no solution, changing to strat 2.
Strat 2 returned no solution, changing to strat 3.
Strat 3 not useful now, changing to strat 4.
using full_word in strat2: w _ l l s e n _ 
Nothing worked!!
Guessing letter: i
Server response: {'game_id': '6e737ad1119c', 'status': 'ongoing', 'tries_remains': 3, 'word': 'w _ l l s e n _ '}
using full_word in strat2: w _ l l s e n _ 
Nothing worked!!
Guessing letter: a
Server response: {'game_id': '6e737ad1119c', 'status': 'ongoing', 'tries_remains': 3, 'word': 'w a l l s e n _ '}
using full_word in strat2: w a l l s e n _ 
Nothing worked!!
Guessing letter: o
Server response: {'game_id': '6e737ad1119c', 'status': 'ongoing', 'tries_remains': 2, 'word': 'w a l l s e n _ '}
using full_word in strat2: w a l l s e n _ 
Nothing worked!!
Guessing letter: c
Server response: {'game_id': '6e737ad1119c', 'status': 'ongoing', 'tries_rema

Server response: {'game_id': '49858ce004d9', 'status': 'success', 'tries_remains': 4, 'word': 'r e o u t f i t t e d '}
Successfully finished game: 49858ce004d9
Successfully start a new game! Game ID: 9b69f9817773. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': '9b69f9817773', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ e _ _ _ '}
Guessing letter: r
Server response: {'game_id': '9b69f9817773', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ e _ _ _ '}
Guessing letter: l
Server response: {'game_id': '9b69f9817773', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ l _ _ _ e _ _ _ '}
Guessing letter: a
Server response: {'game_id': '9b69f9817773', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ l a _ _ e _ _ _ '}
Guessing letter: t
Server response: {'game_id': '9b69f9817773', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ l a _ _ e _ _ _ '}
Guessing letter: f
Server response: {'game_id': '9b69f98177

In [77]:
print(practice_counter)
print(practice_success)

22
[[0, 7, 2, 7, 12], [0, 7, 2, 7, 6]]


In [78]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


run 1913 practice games out of an allotted 100,000. practice success rate so far = 0.327


## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [27]:
for i in range(100):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=True)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
Current Scope is empty, changing to strat2
Playing  1  th game
Current Scope is empty, changing to strat2
Strat 2 returned no solution, changing to strat 3.
Strat 3 not useful now, changing to strat 4.
new_word in strat4 reduction: _ n o c c _ e t t i 
Playing  2  th game
Strat 1 returned no solution, changing to strat 2.
Strat 2 returned no solution, changing to strat 3.
Strat 3 not useful now, changing to strat 4.
new_word in strat4 reduction: l a t i _ u n _ i u _ 
new_word in strat4 reduction: l a t i _ u n d i u _ 
Playing  3  th game
Strat 1 returned no solution, changing to strat 2.
Strat 2 returned no solution, changing to strat 3.
Strat 3 not useful now, changing to strat 4.
new_word in strat4 reduction: c e p h a l _ c h _ r _ a t a 
new_word in strat4 reduction: c e p h a l o c h o r _ a t a 
new_word in strat4 reduction: c e p h a l o c h o r _ a t a 
new_word in strat4 reduction: c e p h a l o c h o r _ a t a 
new_word in strat4 reduction: c e p h a l o

Playing  33  th game
Current Scope is empty, changing to strat2
Playing  34  th game
Strat 1 returned no solution, changing to strat 2.
Strat 2 returned no solution, changing to strat 3.
Strat 3 not useful now, changing to strat 4.
new_word in strat4 reduction: _ i _ e r n i _ i _ e 
new_word in strat4 reduction: _ i _ e r n i _ i _ e 
new_word in strat4 reduction: _ i _ e r n i _ 
new_word in strat4 reduction: h i _ e r n i _ 
new_word in strat4 reduction: h i b e r n i _ 
Playing  35  th game
Strat 1 returned no solution, changing to strat 2.
Strat 2 returned no solution, changing to strat 3.
Strat 3 not useful now, changing to strat 4.
new_word in strat4 reduction: e p i p o _ i _ i _ 
new_word in strat4 reduction: e p i p o _ i _ i _ 
new_word in strat4 reduction: e p i p o _ i _ i _ 
new_word in strat4 reduction: e p i p o _ i t i _ 
new_word in strat4 reduction: e p i p o _ i t i _ 
new_word in strat4 reduction: e p i p o _ i t i _ 
new_word in strat4 reduction: e p i p o _ i t 


new_word in strat4 reduction: r n _ a l l 
Playing  69  th game
Current Scope is empty, changing to strat2
Strat 2 returned no solution, changing to strat 3.
Strat 3 not useful now, changing to strat 4.
new_word in strat4 reduction: h a p _ o _ o n t _ 
new_word in strat4 reduction: h a p l o _ o n t _ 
new_word in strat4 reduction: h a p l o _ o n t _ 
Playing  70  th game
Current Scope is empty, changing to strat2
Strat 2 returned no solution, changing to strat 3.
Playing  71  th game
Playing  72  th game
Playing  73  th game
Strat 1 returned no solution, changing to strat 2.
Strat 2 returned no solution, changing to strat 3.
Strat 3 not useful now, changing to strat 4.
new_word in strat4 reduction: _ e _ i s s e 
new_word in strat4 reduction: _ e _ i s s e 
new_word in strat4 reduction: _ e _ i s s e 
new_word in strat4 reduction: _ e t i s s e 
Playing  74  th game
Playing  75  th game
Playing  76  th game
Current Scope is empty, changing to strat2
Strat 2 returned no solution, cha

In [79]:
counter= 0
success = [[0,0,0,0,0], [0,0,0,0,0]] #fisrt is failure array and second is sucess array
for i in range(81):
    print('Playing ', i, ' th game')
    (res, desc) = api.start_game(practice=0,verbose=True)
    counter += res
    success[res][desc] += 1
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.4)

Playing  0  th game
Successfully start a new game! Game ID: e19a498afbab. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': 'e19a498afbab', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ '}
Guessing letter: i
Server response: {'game_id': 'e19a498afbab', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ i _ _ _ '}
Guessing letter: a
Server response: {'game_id': 'e19a498afbab', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ a _ i a _ _ '}
Guessing letter: r
Server response: {'game_id': 'e19a498afbab', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ a _ i a r _ '}
Guessing letter: g
Server response: {'game_id': 'e19a498afbab', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ a _ i a r _ '}
Guessing letter: d
Server response: {'game_id': 'e19a498afbab', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ a _ i a r d '}
Guessing letter: p
Server response: {'game_id': 'e19a498afbab', 'status': 'ongo

Server response: {'game_id': '8c80722b02db', 'status': 'ongoing', 'tries_remains': 5, 'word': 'a d _ _ d _ _ a _ e d '}
using new_word from strat3 reduction: a d _ _ d _ _ a _ 
Guessing letter: j
Server response: {'game_id': '8c80722b02db', 'status': 'ongoing', 'tries_remains': 5, 'word': 'a d j _ d _ _ a _ e d '}
using new_word from strat3 reduction: a d j _ d _ _ a _ 
Guessing letter: c
Server response: {'game_id': '8c80722b02db', 'status': 'ongoing', 'tries_remains': 5, 'word': 'a d j _ d _ c a _ e d '}
using new_word from strat3 reduction: a d j _ d _ c a _ 
Guessing letter: u
Server response: {'game_id': '8c80722b02db', 'status': 'ongoing', 'tries_remains': 5, 'word': 'a d j u d _ c a _ e d '}
using new_word from strat3 reduction: a d j u d _ c a _ 
Guessing letter: t
Server response: {'game_id': '8c80722b02db', 'status': 'ongoing', 'tries_remains': 5, 'word': 'a d j u d _ c a t e d '}
using new_word from strat3 reduction: a d j u d _ c a t 
Guessing letter: i
Server response: {'g

Server response: {'game_id': 'a4862c3ee2b0', 'status': 'ongoing', 'tries_remains': 4, 'word': 'd i s _ _ s s i o n s '}
using new_word from strat3 reduction: _ _ 
Guessing letter: a
Server response: {'game_id': 'a4862c3ee2b0', 'status': 'ongoing', 'tries_remains': 4, 'word': 'd i s _ a s s i o n s '}
using new_word from strat3 reduction: _ a 
Strat 3 returned no solution, changing to strat 4.
using new_word in clean3 reduction: _ a 
Guessing letter: r
Server response: {'game_id': 'a4862c3ee2b0', 'status': 'ongoing', 'tries_remains': 3, 'word': 'd i s _ a s s i o n s '}
using new_word in clean3 reduction: _ a 
Guessing letter: l
Server response: {'game_id': 'a4862c3ee2b0', 'status': 'ongoing', 'tries_remains': 2, 'word': 'd i s _ a s s i o n s '}
using new_word in clean3 reduction: _ a 
Guessing letter: t
Server response: {'game_id': 'a4862c3ee2b0', 'status': 'ongoing', 'tries_remains': 1, 'word': 'd i s _ a s s i o n s '}
using new_word in clean3 reduction: _ a 
Guessing letter: m
Serv

Server response: {'game_id': 'd896e24226f3', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ _ _ _ _ '}
Guessing letter: i
Server response: {'game_id': 'd896e24226f3', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ _ _ i _ '}
Guessing letter: o
Server response: {'game_id': 'd896e24226f3', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ o _ o _ _ o _ i _ '}
Guessing letter: r
Server response: {'game_id': 'd896e24226f3', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ r o _ o _ _ o r i _ '}
Strat 1 returned no solution, changing to strat 2.
Strat 2 returned no solution, changing to strat 3.
Strat 3 not useful now, changing to strat 4.
using full_word in strat2: _ _ r o _ o _ _ o r i _ 
Nothing worked!!
Guessing letter: a
Server response: {'game_id': 'd896e24226f3', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ r o _ o _ _ o r i _ '}
using full_word in strat2: _ _ r o _ o _ _ o r i _ 
Nothing worked!!
Guessing letter: n
Server res

Server response: {'game_id': '3ac57f58888e', 'status': 'ongoing', 'tries_remains': 4, 'word': 'a _ h i e r '}
Guessing letter: c
Server response: {'game_id': '3ac57f58888e', 'status': 'ongoing', 'tries_remains': 3, 'word': 'a _ h i e r '}
Current Scope is empty, changing to strat2
Strat 2 returned no solution, changing to strat 3.
using new_word from strat3 reduction: a _ h i 
Guessing letter: s
Server response: {'game_id': '3ac57f58888e', 'status': 'success', 'tries_remains': 3, 'word': 'a s h i e r '}
Successfully finished game: 3ac57f58888e
Playing  17  th game
Successfully start a new game! Game ID: 34e69886fed0. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: i
Server response: {'game_id': '34e69886fed0', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ i _ _ _ _ _ _ _ '}
Guessing letter: n
Server response: {'game_id': '34e69886fed0', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ n _ _ _ i _ _ _ n _ _ _ '}
Guessing letter: e
Server resp

...

Server response: {'game_id': 'e67672830f13', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ _ _ i s h e s '}
Guessing letter: l
Server response: {'game_id': 'e67672830f13', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ _ l i s h e s '}
Strat 1 returned no solution, changing to strat 2.
Guessing letter: v
Server response: {'game_id': 'e67672830f13', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ _ _ l i s h e s '}
Guessing letter: m
Server response: {'game_id': 'e67672830f13', 'status': 'ongoing', 'tries_remains': 1, 'word': '_ _ _ l i s h e s '}
Guessing letter: b
Server response: {'game_id': 'e67672830f13', 'status': 'ongoing', 'tries_remains': 1, 'word': '_ b _ l i s h e s '}
Strat 2 returned no solution, changing to strat 3.
using new_word from strat3 reduction: _ b _ l 
Guessing letter: a
Server response: {'game_id': 'e67672830f13', 'status': 'ongoing', 'tries_remains': 1, 'word': 'a b _ l i s h e s '}
using new_word from strat3 reduction: a b _ l 
Guessing lett

Server response: {'game_id': '6dab491bd823', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ _ i _ _ _ _ _ _ _ '}
Guessing letter: e
Server response: {'game_id': '6dab491bd823', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ _ i _ _ _ _ e _ _ '}
Guessing letter: n
Server response: {'game_id': '6dab491bd823', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ n _ _ _ _ _ _ _ i _ _ _ n e _ _ '}
Guessing letter: s
Server response: {'game_id': '6dab491bd823', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ n _ _ _ _ _ _ _ i _ _ _ n e s s '}
Guessing letter: l
Server response: {'game_id': '6dab491bd823', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ n _ _ _ _ _ _ _ i _ _ l n e s s '}
Guessing letter: c
Server response: {'game_id': '6dab491bd823', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ n _ _ _ _ _ _ _ i c _ l n e s s '}
Guessing letter: t
Server response: {'game_id': '6dab491bd823', 'status': 'ongoing', 'tries_remains': 5, 'word': 

Server response: {'game_id': 'ae3a5f139026', 'status': 'ongoing', 'tries_remains': 1, 'word': 'h i _ h p i t c h e d '}
using new_word from strat3 reduction: h i _ h p i t c h 
Guessing letter: g
Server response: {'game_id': 'ae3a5f139026', 'status': 'success', 'tries_remains': 1, 'word': 'h i g h p i t c h e d '}
Successfully finished game: ae3a5f139026
Playing  70  th game
Successfully start a new game! Game ID: 096a3aea72cb. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': '096a3aea72cb', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ e _ _ e e _ e _ '}
Guessing letter: d
Server response: {'game_id': '096a3aea72cb', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ e _ _ e e _ e d '}
Guessing letter: l
Server response: {'game_id': '096a3aea72cb', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ e _ _ e e _ e d '}
Guessing letter: t
Server response: {'game_id': '096a3aea72cb', 'status': 'ongoing', 'tr

Server response: {'game_id': 'b4e5aa2448b4', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ e _ _ _ _ _ _ '}
Guessing letter: r
Server response: {'game_id': 'b4e5aa2448b4', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ e _ _ _ _ _ _ '}
Guessing letter: l
Server response: {'game_id': 'b4e5aa2448b4', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ e _ _ _ l _ _ '}
Guessing letter: a
Server response: {'game_id': 'b4e5aa2448b4', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ a _ _ _ e _ _ _ l _ _ '}
Guessing letter: d
Server response: {'game_id': 'b4e5aa2448b4', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ a _ _ _ e _ d _ l _ _ '}
Strat 1 returned no solution, changing to strat 2.
Strat 2 returned no solution, changing to strat 3.
Strat 3 not useful now, changing to strat 4.
using full_word in strat2: _ a _ _ _ e _ d _ l _ _ 
Nothing worked!!
Guessing letter: i
Server response: {'game_id': 'b4e5aa2448b4', 'status': 'ongoing', 'tries_remain

Server response: {'game_id': '5dc425e71b6d', 'status': 'ongoing', 'tries_remains': 5, 'word': 'c a _ _ t i c i _ e '}
using full_word in strat2: c a _ _ t i c i _ e 
Nothing worked!!
Guessing letter: o
Server response: {'game_id': '5dc425e71b6d', 'status': 'ongoing', 'tries_remains': 4, 'word': 'c a _ _ t i c i _ e '}
using full_word in strat2: c a _ _ t i c i _ e 
Nothing worked!!
Guessing letter: r
Server response: {'game_id': '5dc425e71b6d', 'status': 'ongoing', 'tries_remains': 3, 'word': 'c a _ _ t i c i _ e '}
using full_word in strat2: c a _ _ t i c i _ e 
Nothing worked!!
Guessing letter: s
Server response: {'game_id': '5dc425e71b6d', 'status': 'ongoing', 'tries_remains': 3, 'word': 'c a _ s t i c i _ e '}
using full_word in strat2: c a _ s t i c i _ e 
Nothing worked!!
Guessing letter: l
Server response: {'game_id': '5dc425e71b6d', 'status': 'ongoing', 'tries_remains': 2, 'word': 'c a _ s t i c i _ e '}
using full_word in strat2: c a _ s t i c i _ e 
Nothing worked!!
Guessing 

In [80]:
print(counter)
print(success)

28
[[0, 9, 9, 7, 28], [0, 8, 1, 8, 11]]


## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [81]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

overall success rate = 0.377


In [82]:
total_recorded_runs

700